# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
from sqlalchemy import create_engine
import pandas as pd
import numpy as np

import nltk
nltk.download(['punkt', 'wordnet'])

from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline

from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import classification_report

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('Disaster', con=engine)
X = df['message'].values
Y = df.iloc[:, 4:].values


### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    tokens = word_tokenize(text)
    
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()

    # iterate through each token
    clean_tokens = []
    for tok in tokens:
        
        # lemmatize, normalize case, and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

pipeline.fit(X_train, Y_train)

y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
labels = ['related', 'request', 'offer', 'aid_related', 'medical_help',
  'medical_products', 'search_and_rescue', 'security', 'military',
  'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
  'missing_people', 'refugees', 'death', 'other_aid', 'infrastructure_related',
  'transport', 'buildings', 'electricity', 'tools', 'hospitals', 'shops',
  'aid_centers', 'other_infrastructure', 'weather_related', 'floods', 'storm',
  'fire', 'earthquake', 'cold', 'other_weather', 'direct_report']

print(classification_report(Y_test, y_pred, target_names=labels))

                        precision    recall  f1-score   support

               related       0.77      0.93      0.84      5026
               request       0.43      0.08      0.14      1065
                 offer       0.00      0.00      0.00        32
           aid_related       0.43      0.20      0.27      2702
          medical_help       0.05      0.00      0.00       490
      medical_products       0.11      0.01      0.01       324
     search_and_rescue       0.00      0.00      0.00       169
              security       0.00      0.00      0.00       118
              military       0.00      0.00      0.00       230
           child_alone       0.00      0.00      0.00         0
                 water       0.05      0.00      0.00       426
                  food       0.20      0.01      0.02       726
               shelter       0.08      0.01      0.01       541
              clothing       0.11      0.01      0.02       102
                 money       0.20      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [7]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f83b56aa268>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

### 6. Improve your model
Use grid search to find better parameters. 

In [8]:
parameters = {
    #'vect__ngram_range' : ((1, 2), (2, 2)),
    'clf__estimator__n_jobs': [2, 3, 4]
}

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=12, n_jobs=-1)

cv.fit(X_train, Y_train)

y_pred = cv.predict(X_test)

cv.best_params_

Fitting 3 folds for each of 3 candidates, totalling 9 fits
[CV] clf__estimator__n_jobs=2 ........................................
[CV]  clf__estimator__n_jobs=2, score=0.163003663003663, total= 1.1min
[CV] clf__estimator__n_jobs=2 ........................................


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  1.3min remaining:    0.0s


[CV]  clf__estimator__n_jobs=2, score=0.13965201465201466, total= 1.1min
[CV] clf__estimator__n_jobs=2 ........................................


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:  2.6min remaining:    0.0s


[CV]  clf__estimator__n_jobs=2, score=0.16104411540222865, total= 1.1min
[CV] clf__estimator__n_jobs=3 ........................................


...

[CV]  clf__estimator__n_jobs=4, score=0.1485042735042735, total= 1.1min
[CV] clf__estimator__n_jobs=4 ........................................


[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed: 10.3min remaining:    0.0s


[CV]  clf__estimator__n_jobs=4, score=0.16470767821706608, total= 1.1min


[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed: 11.6min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed: 11.6min finished


{'clf__estimator__n_jobs': 4}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [9]:
print(classification_report(Y_test, y_pred, target_names=labels))

                        precision    recall  f1-score   support

               related       0.78      0.93      0.85      5026
               request       0.43      0.08      0.13      1065
                 offer       0.00      0.00      0.00        32
           aid_related       0.46      0.20      0.28      2702
          medical_help       0.13      0.00      0.01       490
      medical_products       0.14      0.01      0.02       324
     search_and_rescue       0.00      0.00      0.00       169
              security       0.00      0.00      0.00       118
              military       0.25      0.00      0.01       230
           child_alone       0.00      0.00      0.00         0
                 water       0.09      0.01      0.01       426
                  food       0.16      0.01      0.02       726
               shelter       0.06      0.00      0.01       541
              clothing       0.33      0.01      0.02       102
                 money       0.25      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [10]:
from sklearn.ensemble import AdaBoostClassifier

pipeline_ada = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])

pipeline_ada.fit(X_train, Y_train)

y_pred_ada = pipeline_ada.predict(X_test)

In [11]:
print(classification_report(Y_test, y_pred_ada, target_names=labels))

                        precision    recall  f1-score   support

               related       0.77      0.98      0.86      5026
               request       0.44      0.12      0.19      1065
                 offer       0.33      0.03      0.06        32
           aid_related       0.46      0.14      0.22      2702
          medical_help       0.06      0.00      0.00       490
      medical_products       0.12      0.00      0.01       324
     search_and_rescue       0.00      0.00      0.00       169
              security       0.00      0.00      0.00       118
              military       0.00      0.00      0.00       230
           child_alone       0.00      0.00      0.00         0
                 water       0.00      0.00      0.00       426
                  food       0.21      0.01      0.01       726
               shelter       0.00      0.00      0.00       541
              clothing       0.00      0.00      0.00       102
                 money       0.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [12]:
pipeline_ada.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f83b56aa268>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
             learning_rate=1.0, n_estimators=50, random_state=None),
              n_jobs=1))],
 'vect': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_features=None, min_df=1,
         ngram_rang

In [13]:
parameters_ada = {
    #'clf__estimator__learning_rate' : [1.0, 2.0],
    'clf__estimator__n_estimators': [60, 70, 80]
}

cv_ada_grid = GridSearchCV(pipeline_ada, param_grid=parameters_ada, verbose=12, n_jobs=-1)

cv_ada_grid.fit(X_train, Y_train)

y_pred_ada_grid = cv_ada_grid.predict(X_test)

Fitting 3 folds for each of 3 candidates, totalling 9 fits
[CV] clf__estimator__n_estimators=60 .................................
[CV]  clf__estimator__n_estimators=60, score=0.15018315018315018, total= 2.1min
[CV] clf__estimator__n_estimators=60 .................................


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  2.3min remaining:    0.0s


[CV]  clf__estimator__n_estimators=60, score=0.13934676434676435, total= 2.1min
[CV] clf__estimator__n_estimators=60 .................................


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:  4.6min remaining:    0.0s


[CV]  clf__estimator__n_estimators=60, score=0.15295374751946267, total= 2.1min
[CV] clf__estimator__n_estimators=70 .................................


...

[Parallel(n_jobs=-1)]: Done   7 out of   7 | elapsed: 17.6min remaining:    0.0s


[CV]  clf__estimator__n_estimators=80, score=0.1343101343101343, total= 2.7min
[CV] clf__estimator__n_estimators=80 .................................


[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed: 20.6min remaining:    0.0s


[CV]  clf__estimator__n_estimators=80, score=0.14516867653793314, total= 2.7min


[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed: 23.5min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed: 23.5min finished


In [14]:
print(classification_report(Y_test, y_pred_ada_grid, target_names=labels))

                        precision    recall  f1-score   support

               related       0.77      0.98      0.86      5026
               request       0.42      0.12      0.19      1065
                 offer       0.00      0.00      0.00        32
           aid_related       0.47      0.15      0.22      2702
          medical_help       0.06      0.00      0.00       490
      medical_products       0.11      0.00      0.01       324
     search_and_rescue       0.00      0.00      0.00       169
              security       0.00      0.00      0.00       118
              military       0.22      0.01      0.02       230
           child_alone       0.00      0.00      0.00         0
                 water       0.00      0.00      0.00       426
                  food       0.23      0.01      0.02       726
               shelter       0.00      0.00      0.00       541
              clothing       0.08      0.01      0.02       102
                 money       0.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [15]:
cv_ada_grid.best_params_

{'clf__estimator__n_estimators': 60}

### 9. Export your model as a pickle file

In [ ]:
import pickle

pickle.dump(cv_ada_grid, open('ada_model','wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.